In [1]:
from utils import computeR2
from utils import computeR2_train
from utils import computeR2_train_self
from utils import computeR2_unlabel
import numpy as np
from tqdm import tqdm
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import pdist, squareform
from utils import get_input
from utils import get_input_all
from utils import normalized_data
from Save_data import save_data
from Plot_performance import plot_performance
import pickle
import numpy as np
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings('ignore')
from utils import feature_selection

In [3]:
# Hyperparameters
numDataTotal = 509
labeledPoolN = 10
batchSz = 1
nAccs = (numDataTotal-labeledPoolN)//batchSz
RepeatTimes = 1
runs=1
fs_score=0.98
freq=10
Alg='GSx_fs'
# Get the input files
myfile =r'Data/CombinedPSP.csv'
# X, y = get_input(myfile)
# If combine feature selection
X, y = get_input_all(myfile)

In [9]:
R2Smooth = []
MSEsmooth = []
MAEsmooth = []
Infosmooth = []
R2_train = []
R2_trainS = []
SelectData = []

for rt in tqdm(range(runs)):
    np.random.seed(rt)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=rt)
    dataPool = pd.concat([X_train, y_train], axis=1)
    SelectIdx=np.random.choice(dataPool.index, labeledPoolN, replace=False)
    dataPoolL = dataPool.iloc[SelectIdx, :]
    dataPool = dataPool.drop(SelectIdx)
    
    data = pd.concat([X_train, y_train], axis=1)
    distX = squareform(pdist(data.iloc[:,0:-1]))
    
    Idx = []
    Idx = SelectIdx.tolist()
    idsTest=np.arange(0,len(y_train))
    idsTest=np.delete(idsTest,Idx)

    R2Res = np.empty((0,1), float)
    MSERes = np.empty((0,1), float)
    MAERes = np.empty((0,1), float)
    InfoRes = np.empty((0,1), float)
    R2Res_t = np.empty((0,1), float)
    R2Res_tS = np.empty((0,1), float)

    R2, Model, MSEstart, MAEstart = computeR2(dataPoolL, X_test, y_test, fs=True)
    R2Res = np.append(R2Res, R2, axis=0)
    MSERes = np.append(MSERes, MSEstart, axis=0)
    MAERes = np.append(MAERes, MAEstart, axis=0)
    Info = computeR2_unlabel(dataPool, dataPoolL, Model, fs=True)
    InfoRes = np.append(InfoRes, Info, axis=0)

    R2_t, Model, MSEstart_t,_ = computeR2_train(dataPoolL, X_train, y_train, fs=True)
    R2Res_t = np.append(R2Res_t, R2_t, axis=0)

    R2_tS, ModelS, MSEstart_tS,_ = computeR2_train_self(dataPoolL, fs=True)
    R2Res_tS = np.append(R2Res_tS, R2_tS, axis=0)

    # feature selection
    indices = feature_selection(dataPoolL.iloc[:, 0:-1],dataPoolL.iloc[:, -1], fs_score, 0, Alg)
    dataPoolL_fs = pd.concat([dataPoolL.iloc[:, 0:-1].iloc[:, indices],dataPoolL.iloc[:, -1]],axis=1)
    dataPool_fs = pd.concat([dataPool.iloc[:, 0:-1].iloc[:, indices], dataPool.iloc[:, -1]],axis=1)
    data_fs = pd.concat([data.iloc[:, 0:-1].iloc[:, indices], data.iloc[:, -1]],axis=1)

    # get model with fewer features
    _, Model_fs, _, _ = computeR2(dataPoolL_fs, X_test.iloc[:, indices], y_test, fs=True)

    for n in np.arange(10,509):
        distX = squareform(pdist(data_fs.iloc[:,0:-1]))
        # dist=distX[np.ix_(idsTest, np.array(Idx[0:n]))].min(axis=1)
        # idx=np.argmax(dist)
        # Idx.append(idsTest[idx])
        # idsTest=np.delete(idsTest,idx)

        distY=np.zeros((dataPool_fs.iloc[:,0:-1].shape[0],n))

        for i in np.arange(n):
            distY[:,i]= abs(Model_fs.predict(dataPool_fs.iloc[:,0:-1])-dataPoolL_fs.iloc[i,-1]*np.ones((dataPool_fs.iloc[:,0:-1].shape[0])))

        a1=distX[np.ix_(idsTest, np.array(Idx[0:n]))]
        a2=distY
        dist=(np.multiply(a1, a2)).min(axis=1)

        idx2=np.argmax(dist)
        Idx.append(idsTest[idx2])
        idsTest=np.delete(idsTest,idx2)

        databatch_fs=dataPool_fs.iloc[idx2,:].to_frame().T
        dataPool_fs=data_fs.iloc[idsTest,:]
        dataPoolL_fs = pd.concat([dataPoolL_fs, databatch_fs], axis=0)
        
        databatch=dataPool.iloc[idx2,:].to_frame().T
        dataPool=data.iloc[idsTest,:]
        dataPoolL = pd.concat([dataPoolL, databatch], axis=0)

        cR2, Model, cMSE, cMAE = computeR2(dataPoolL, X_test, y_test, fs=True)
        R2Res = np.append(R2Res, cR2, axis=0)
        MSERes = np.append(MSERes, cMSE, axis=0)
        MAERes = np.append(MAERes, cMAE, axis=0)
        cInfo = computeR2_unlabel(dataPool, dataPoolL, Model, fs=True)
        InfoRes = np.append(InfoRes, cInfo, axis=0)

        cR2_t, Model, cMSEstart_t,_ = computeR2_train(dataPoolL, X_train, y_train, fs=True)
        R2Res_t = np.append(R2Res_t, cR2_t, axis=0)

        cR2_tS, ModelS, cMSEstart_tS,_ = computeR2_train_self(dataPoolL, fs=True)
        R2Res_tS = np.append(R2Res_tS, cR2_tS, axis=0)

        if n % freq == 0:
            # feature selection
            indices = feature_selection(dataPoolL.iloc[:, 0:-1],dataPoolL.iloc[:, -1], fs_score, 0, Alg)
            dataPoolL_fs = pd.concat([dataPoolL.iloc[:, 0:-1].iloc[:, indices],dataPoolL.iloc[:, -1]],axis=1)
            dataPool_fs = pd.concat([dataPool.iloc[:, 0:-1].iloc[:, indices], dataPool.iloc[:, -1]],axis=1)
            data_fs = pd.concat([data.iloc[:, 0:-1].iloc[:, indices], data.iloc[:, -1]],axis=1)

            # get model with fewer features
            _, Model_fs, _,_ = computeR2(dataPoolL_fs, X_test.iloc[:, indices], y_test, fs=True)

    R2Smooth.append(R2Res)
    MSEsmooth.append(MSERes)
    MAEsmooth.append(MAERes)
    Infosmooth.append(InfoRes)
    R2_train.append(R2Res_t)
    R2_trainS.append(R2Res_tS)
    SelectData.append(dataPoolL)
    
#     SelectData.append(dataPoolL)
R2Smooth = np.asarray(R2Smooth)
MSEsmooth = np.asarray(MSEsmooth)
MAEsmooth = np.asarray(MAEsmooth)
InfoSmooth = np.asarray(Infosmooth)
R2_train = np.asarray(R2_train)
R2_trainS = np.asarray(R2_trainS)

  0%|          | 0/1 [00:00<?, ?it/s]

Features sorted by their score:
[(0.1431, 'd11'), (0.1402, 'd3'), (0.1222, 'd14'), (0.1141, 'd21'), (0.0857, 'd18'), (0.079, 'd19'), (0.0658, 'd7'), (0.0505, 'd16'), (0.0414, 'd1'), (0.0305, 'd13'), (0.0287, 'd2'), (0.0232, 'd5'), (0.022, 'd10'), (0.0208, 'd4'), (0.016, 'd15'), (0.0098, 'd9'), (0.0024, 'd20'), (0.0016, 'd6'), (0.0015, 'd12'), (0.0007, 'd8'), (0.0007, 'd17')]
Features sorted by their score:
[(0.1428, 'd3'), (0.1415, 'd16'), (0.1231, 'd11'), (0.1149, 'd18'), (0.1075, 'd19'), (0.0788, 'd1'), (0.0472, 'd21'), (0.0385, 'd2'), (0.0324, 'd15'), (0.0305, 'd7'), (0.0295, 'd8'), (0.0225, 'd10'), (0.0206, 'd14'), (0.0196, 'd5'), (0.0195, 'd4'), (0.0102, 'd17'), (0.01, 'd9'), (0.0042, 'd20'), (0.0028, 'd12'), (0.0022, 'd13'), (0.0019, 'd6')]
Features sorted by their score:
[(0.3034, 'd3'), (0.2758, 'd11'), (0.1332, 'd2'), (0.0725, 'd19'), (0.0579, 'd8'), (0.0333, 'd18'), (0.0226, 'd15'), (0.0182, 'd21'), (0.0095, 'd6'), (0.0084, 'd20'), (0.0077, 'd4'), (0.0076, 'd5'), (0.0073, 'd9

100%|██████████| 1/1 [19:49<00:00, 1189.96s/it]
